In [32]:
# import modules
import geopandas as gpd
import matplotlib.pyplot as plt
#from shapely import wkt
from shapely.geometry import Point
#import shapely
import osmnx as ox
import pandas as pd
import networkx as nx
from shapely.geometry import LineString
from tabulate import tabulate
import math as math
from geojson import Feature, LineString, FeatureCollection
import random
import statistics
import matplotlib.pyplot as plt
from scipy.stats import norm
import numpy as np




In [33]:
# Reading csv files into panda DataFrames
dfNodes = pd.read_csv("/Users/suvihanninen/Desktop/Research_project/CoolWalk/data_Nottingham/nottingham_nodes.csv")
dfEdges = pd.read_csv("/Users/suvihanninen/Desktop/Research_project/CoolWalk/data_Nottingham/nottingham_edges.csv")


In [34]:
# Reading csv files into panda DataFrames
#dfNodes = pd.read_csv("/mnt/c/users/kriso/documents/itu/5th_semester/research_project/CoolWalk/data/CoolWalksDataStorge/nodes.csv")
#dfEdges = pd.read_csv("/mnt/c/users/kriso/documents/itu/5th_semester/research_project/CoolWalk/data/CoolWalksDataStorge/edges.csv")

In [35]:
#Dont execute

#TODO If shadow is None it is 0. However, our current cost-function is not compatible with this, since it is a division, and will thus result in division by 0 errors.
#   TEMPORARY SOLUTION: Dont execute this cell. Let the shortest_path_algorithm cost-function fill out the None shadow values with 1. 

#Find and replace all None-valued 'shadowed_length' instances in edges with 0
#dfEdges['shadowed_length'] = dfEdges['shadowed_length'].fillna(0)

In [36]:
# Find and replace all None-valued 'full_length' instances in edges with the average full_length value
#(dfEdges['full_length'].mean())

In [37]:
# Find and discard helper nodes from dfNodes. Helper nodes do not have a 'geopoint' attribute value in the dataframe
    # Only rows with helper=Nan should remain
dfNodes.drop(dfNodes[dfNodes.osm_id == 0].index, inplace=True)
#print(tabulate(gdfNodes, headers='keys', tablefmt='psql'))

In [38]:
# Find and discard helper edges from gdfEdges
    # Only rows with helper=NaN should remain
dfEdges.drop(dfEdges[dfEdges.helper == True].index, inplace=True)
#print(tabulate(gdfEdges, headers='keys', tablefmt='psql'))

In [39]:
# Setting osmid as index for nodes (mutating the dataframe). ATTENTION! the osmnx code will create the x_lookup and y_lookup dictionaries based on the index of the nodes data frame. That is, the keys of these dictionaries will consist
# of the values of the index column, and the values will be the lon(x_lookup) and lat(y_lookup) values respectively   

dfNodes.set_index('vertex_id', inplace=True) 
#dfNodes.set_index('osm_id', inplace=True) 



#Printing dataframes in readable format
#print(tabulate(gdfNodes, headers='keys', tablefmt='psql'))

In [40]:
#setting src_id, dst_id and osm_id of edge geodf´s as the multiindex(u, v, key) of each node (mutating the dataframe)
dfEdges.set_index(['src_id','dst_id','osm_id'], inplace=True) #should we use the osm_id from the Nodes instead, since Henrik says they have 0 values to indicate multi-edges, which is what the value 'k' is intended for in the documentation of utils_graph.graph_from_gdfs

#Printing dataframes in readable format
#print(tabulate(gdfEdges, headers='keys', tablefmt='psql'))

In [41]:
# Transforming relevant columns of DataFrames into GeoSeries to be used as geometry column of GeoDataFrames
dfNodes['pointgeom'] = gpd.GeoSeries.from_wkt(dfNodes['pointgeom'])
dfEdges['edgegeom'] = gpd.GeoSeries.from_wkt(dfEdges['edgegeom'])

In [42]:
# Defining GeoDataFrames with GeoSeries columns from above
gdfNodes = gpd.GeoDataFrame(dfNodes, geometry='pointgeom', crs="EPSG:27700")
gdfEdges = gpd.GeoDataFrame(dfEdges, geometry='edgegeom', crs="EPSG:27700")

In [43]:
# Data wrangling: Editing/andjustment GeoDataFrames
gdfNodes.rename(columns = {'lat':'x', 'lon':'y'}, inplace = True)
gdfNodes["x"] = gdfNodes["pointgeom"].x
gdfNodes["y"] = gdfNodes["pointgeom"].y

#print(tabulate(gdfEdges, headers='keys', tablefmt='psql'))
#print(gdfEdges)



In [44]:
G = ox.utils_graph.graph_from_gdfs(gdfNodes, gdfEdges, graph_attrs=None)

In [45]:
def weigth_function_original(u, v, d):
    assert len(list(attr.get('full_length', 1) for attr in d.values())) == 1 #the graph only have one sttribute with name = 'full_length'
    dict = list(d.values())[0].get('full_length')  # w could perhaps also just access full_length and shadowed_length in this way?
    if dict is not None : #apparently some edges does not have a 'full_length' attribute, and in that case the attribute value will be none. check in the data frame for none values
        print(dict)
    full_length = min(attr.get('full_length', 1) for attr in d.values()) #if 'full_length' value is None, then it will default to 1... keyError will return None because we use the get() method
    shadow_length = min(attr.get('shadowed_length', 1) for attr in d.values())
    result = full_length/shadow_length # we get many 1´s, which either because they default to 1, or because shadow_length==full_length(very unlikely). So in the next line, we filter out all the 1´s to make sure that we actually have real meaningfull values
    if not math.isclose(result, 1): #
        print(result)
    return result

In [113]:
def weight_function_1(u, v, d):
    #print(d.values())
    assert len(list(attr.get('full_length', 1) for attr in d.values())) == 1 #the graph only have one sttribute with name = 'full_length'
    dict = list(d.values())[0].get('full_length')  # w could perhaps also just access full_length and shadowed_length in this way?
    #if dict is not None : #apparently some edges does not have a 'full_length' attribute, and in that case the attribute value will be none. check in the data frame for none values
    #    print(dict)
    full_length = min(attr.get('full_length', 1) for attr in d.values()) #if 'full_length' value is None, then it will default to 1... keyError will return None because we use the get() method
    shadow_length = min(attr.get('shadowed_length', 1) for attr in d.values())
    if shadow_length == 0 :
       return full_length

    result = (shadow_length/full_length) + (full_length/shadow_length) # we get many 1´s, which either because they default to 1, or because shadow_length==full_length(very unlikely). So in the next line, we filter out all the 1´s to make sure that we actually have real meaningfull values
    #if not math.isclose(result, 1): #
    #    print(result)

    #full_length = min(attr.get('full_length', 0) for attr in d.values()) 
  
    # print("scr = ",end='') 
    # print(u)    
    # print("dst = ",end='') 
    # print(v)
    # print("full_length = ",end='') 
    # print(full_length)
    # print("shadow_length = ",end='')
    # print(shadow_length)
    # print("cost = ",end='')
    # print(result)
    # print()
    return result

In [47]:
def weight_function_2(u, v, d):
    #print(d.values())
    assert len(list(attr.get('full_length', 0) for attr in d.values())) == 1 #the graph only have one sttribute with name = 'full_length'
    dict = list(d.values())[0].get('full_length')  # w could perhaps also just access full_length and shadowed_length in this way?
    #if dict is not None : #apparently some edges does not have a 'full_length' attribute, and in that case the attribute value will be none. check in the data frame for none values
    #    print(dict)
    full_length = min(attr.get('full_length', 0) for attr in d.values()) #if 'full_length' value is None, then it will default to 1... keyError will return None because we use the get() method
    shadow_length = min(attr.get('shadowed_length', 0) for attr in d.values())
    sunny_length = full_length-shadow_length
    result = (full_length*0.8) + (sunny_length*1.2) # Applying statistical weights to sunny length and full length
    #if not math.isclose(result, 1): #
    #    print(result)

    # print("scr = ",end='') 
    # print(u)    
    # print("dst = ",end='') 
    # print(v)
    # print("full_length = ",end='') 
    # print(full_length)
    # print("shadow_length = ",end='')
    # print(shadow_length)
    # print("cost = ",end='')
    # print(result)
    # print()

    return result

In [48]:
optimal_path_test1 = nx.shortest_path(G, source=24987, target=216, weight=weight_function_1)

In [49]:
optimal_path_test1 = nx.shortest_path(G, source=24987, target=216, weight=weight_function_2)

In [50]:
regular_shortest_path = nx.shortest_path(G, source=24987, target=216, weight='full_length')

In [51]:
#Graph inspection/extraction

# Extracting the attributes of a given edge(u,v) only by providing scr_id and dst_id. The edge data is given as dict. Afterwards specifying which attribute value to get.
def get_src_dst_distance(src, dst):
     edge_data = G.get_edge_data(src,dst) #ATTENTION! 3rd argument of this function is what should be returned in case edge(u,v) is not found. It defaults to a None object, and thus we put this statement i a try statement to handle such a case
     #print(edge_data)
     try:
          return (min(attr.get('full_length', 0) for attr in edge_data.values())) # Should the value of the specified attribute be None, then the returned value will default to 0
     except AttributeError: 
         print("edge with given scr and dst does not exist in data set")

In [52]:
#Graph inspection/extraction

# Extracting the attributes of a given edge(u,v) only by providing scr_id and dst_id. The edge data is given as dict. Afterwards specifying which attribute value to get.
def get_src_dst_shadowed_distance(src, dst):
     edge_data = G.get_edge_data(src,dst) #ATTENTION! 3rd argument of this function is what should be returned in case edge(u,v) is not found. It defaults to a None object, and thus we put this statement i a try statement to handle such a case
     #print(edge_data)
     try:
          return (min(attr.get('shadowed_length', 0) for attr in edge_data.values())) # Should the value of the specified attribute be None, then the returned value will default to 0
     except AttributeError: 
         print("edge with given scr and dst does not exist in data set")

In [98]:
# Function for getting the total length of a path
#Here we get the length of a path
def get_total_path_length(path):
    try:
        total_length = 0
        for index,scr in enumerate(path):
            if(index==len(path)-1):
                break
            dst = path[index+1]
            scr_dst_distance = get_src_dst_distance(scr, dst)
            total_length += scr_dst_distance
        return total_length
    except TypeError:
        print("No distance between given scr and dst")


In [99]:
#Length of the shadow

def get_total_path_shadowed_length(path):
    try:
        total_length = 0
        for index,scr in enumerate(path):
            if(index==len(path)-1):
                break
            dst = path[index+1]
            scr_dst_distance = get_src_dst_shadowed_distance(scr, dst)
            total_length += scr_dst_distance
        return total_length
    except TypeError:
        print("No distance between given scr and dst")

In [100]:
# Testing 

# Function for getting the fraction of a path which is covered by shadow

def get_shadowed_part_of_path(path):
    try:
        shadowed_part = get_total_path_shadowed_length(path) / get_total_path_length(path)
        return shadowed_part
    except TypeError:
        print("No distance between given scr and dst")


In [56]:
###Generate 80 paths and append them to a list. After calculate 
# - get_shadowed_part_of_path
# - get_total_path_shadowed_length
# - get_total_path_length

# Append these values to a list
# Calculate Mean, Median and Mode

In [108]:
def get_80_paths(srcList, destList, cost):
    list_of_paths = []

    for i in range(5):
    
        optimal_path_test1 = nx.shortest_path(G, srcList[i], destList[i], weight=cost)
        list_of_paths.append(optimal_path_test1)
    
    return list_of_paths

In [101]:
def get_list_of_shadowed_part_of_path(list_of_paths):
    calc_values_list = []
    for onePath in list_of_paths:
        calc_values_list.append(get_shadowed_part_of_path(onePath))
    return calc_values_list

def get_list_of_get_total_path_shadowed_length(list_of_paths):
    calc_values_list = []
    for onePath in list_of_paths:
        calc_values_list.append(get_total_path_shadowed_length(onePath))
    return calc_values_list

def get_list_of_total_path_length(list_of_paths):
    calc_values_list = []
    for onePath in list_of_paths:
        calc_values_list.append(get_total_path_length(onePath))
    return calc_values_list

In [59]:
def get_mean(list):
    mean = statistics.mean(list) 
    return mean

def get_median(list):    
    median = statistics.median(list)
    return median

def get_stdev(list):
    stdev = statistics.stdev(list)
    return stdev

In [60]:
def create_df(data, df_columns):
    df = pd.DataFrame(data, columns = df_columns)
    return df
    

In [64]:
src = random.randint(1, 25648) #inclucive
dest = random.randint(1, 25648)

srcList = []
destList =[]

for oneIter in range(5):
    src = random.randint(1, 25648) #inclucive
    dest = random.randint(1, 25648)
    srcList.append(src)
    destList.append(dest)


[23759, 421, 11131, 4449, 23434, 13078, 19758, 15610, 25329, 11625, 16645, 25265, 20633, 12200, 13112, 2411, 9170, 9051, 3130, 171, 17072, 1117, 9123, 12363, 20753, 5342, 1481, 13462, 20049, 16327, 14105, 22912, 22001, 22531, 8546, 25203, 21724, 20713, 1306, 10134, 332, 20860, 21521, 16154, 2721, 11095, 9678, 10413, 17278, 22345, 12251, 18216, 19830, 11459, 20158, 19539, 3150, 9372, 25210, 3200, 10792, 5540, 1173, 18630, 1288, 6686, 4542, 24882, 18243, 14526, 18302, 3299, 5172, 4446, 4554, 10910, 13868, 12001, 15692, 16172]
[12333, 1864, 9415, 24600, 8819, 21754, 11605, 22279, 7781, 12872, 8717, 24371, 15626, 23859, 21628, 23382, 4115, 8913, 12291, 13736, 8500, 7074, 18138, 22729, 808, 18152, 18605, 20399, 14868, 7064, 16202, 3094, 2801, 15564, 9617, 19431, 20323, 12860, 16802, 20549, 23241, 13337, 21300, 11553, 22997, 21405, 19351, 24270, 7196, 19978, 12108, 8814, 10860, 16954, 12169, 2749, 12443, 14277, 5151, 25517, 924, 14784, 9210, 417, 6276, 15548, 15432, 15147, 9107, 14893, 16691

In [71]:

def getRatio(list_total_length_shortest, list_total_length_optimal):
    ratioList =[]
    for i in range(5):
         ratio = list_total_length_shortest[i]/list_total_length_optimal[i]
         ratioList.append(ratio)
    return ratioList

In [75]:


list_of_paths_short = get_80_paths(srcList, destList, "full_length")
list_shadow_part_shortest = get_list_of_shadowed_part_of_path(list_of_paths_short)
list_shadow_length_shortest = get_list_of_get_total_path_shadowed_length(list_of_paths_short)
list_total_length_shortest = get_list_of_total_path_length(list_of_paths_short)
#Mean
mean_part_shortest = get_mean(list_shadow_part_shortest)
mean_shadow_length_shortest = get_mean(list_shadow_length_shortest)
mean_total_length_shortest = get_mean(list_total_length_shortest)
#Median
median_shortest = get_median(list_shadow_part_shortest)
median_shadow_length_shortest = get_median(list_shadow_length_shortest)
median_total_length_shortest = get_median(list_total_length_shortest)
#Stdev
stdev_part_shortest = get_stdev(list_shadow_part_shortest)
stdev_shadow_length_shortest = get_stdev(list_shadow_length_shortest)
stdev_total_length_shortest = get_stdev(list_total_length_shortest)


###Cost_function_1
list_of_paths_cost1 = get_80_paths(srcList, destList, weight_function_1)
list_shadow_part_cost1 = get_list_of_shadowed_part_of_path(list_of_paths_cost1)
list_shadow_length_cost1 = get_list_of_get_total_path_shadowed_length(list_of_paths_cost1)
list_total_length_cost1 = get_list_of_total_path_length(list_of_paths_cost1)
#Mean
mean_shadow_part_cost1 = get_mean(list_shadow_part_cost1)
mean_shadow_length_cost1 = get_mean(list_shadow_length_cost1)
mean_total_length_cost1 = get_mean(list_total_length_cost1)
#Median
median_shadow_part_cost1 = get_median(list_shadow_part_cost1)
median_shadow_length_cost1 = get_median(list_shadow_length_cost1)
median_cost1 = get_median(list_shadow_part_cost1)
#Stdev
stdev_shadow_part_cost1 = get_stdev(list_shadow_part_cost1)
stdev_shadow_length_cost1 = get_stdev(list_shadow_length_cost1)
stdev_total_length_cost1 = get_stdev(list_total_length_cost1)


###Cost_function_2
list_of_paths_cost2 = get_80_paths(srcList, destList, weight_function_2)
list_shadow_part_cost2 = get_list_of_shadowed_part_of_path(list_of_paths_cost2)
list_shadow_length_cost2 = get_list_of_get_total_path_shadowed_length(list_of_paths_cost2)
list_total_length_cost2 = get_list_of_total_path_length(list_of_paths_cost2)
#mean
mean_shadow_part_cost2 = get_mean(list_shadow_part_cost2)
mean_shadow_length_cost2 = get_mean(list_shadow_length_cost2)
mean_total_length_cost2 = get_mean(list_total_length_cost2)
#median
median_part_cost2 = get_median(list_shadow_part_cost2)
median_shadow_length_cost2 = get_median(list_shadow_length_cost2)
median_total_length_cost2 = get_median(list_total_length_cost2)

#stdev
stdev_part_cost2 = get_stdev(list_shadow_part_cost2)
stdev_shadow_length_cost2 = get_stdev(list_shadow_length_cost2)
stdev_total_length_cost2 = get_stdev(list_total_length_cost2)

#ratio
#shortest path length/cost1/2 path length
#How much longer is shadow path length compared to the shortest path length
ratio_list_cost1 = getRatio(list_total_length_shortest, list_total_length_cost1)
mean_ratio_cost1 = get_mean(ratio_list_cost1)
ratio_list_cost2 = getRatio(list_total_length_shortest, list_total_length_cost2)
mean_ratio_cost2 = get_mean(ratio_list_cost2)



data = [['Shadow part (%)', mean_part_shortest*100, mean_shadow_part_cost1*100, mean_shadow_part_cost2*100], 
         ['Shadow length (m)', mean_shadow_length_shortest, mean_shadow_length_cost1,mean_shadow_length_cost2], 
         ['Full length (m)', mean_total_length_shortest, mean_total_length_cost1, mean_total_length_cost2],
         ['Ratio shortest/optimal (%)', "-", mean_ratio_cost1, mean_ratio_cost2]]

df_columns = ['Mean', 'Shortest Path', 'Optimal Shadow Path: cost_function_1','Optimal Shadow Path: cost_function_2']

dfMean=create_df(data, df_columns)


In [81]:
#Make CSV with 
#1 column with shortest path, one with shadow path

data = [[list_total_length_shortest, list_total_length_cost1]]

dfData = {'Shortest Path': pd.Series(list_total_length_shortest),
              'Optimal Shadow Path' : pd.Series(list_total_length_cost1)}

dfPaths=pd.DataFrame(dfData)

dfPaths.to_csv("path_comparison.csv", index = False)

In [82]:
dfPaths

,Shortest Path,Optimal Shadow Path
0,10658.005635,15059.077107
1,9309.035910,12622.240546
2,19542.870758,21803.966318
3,6044.435014,6921.931229
4,7072.767177,11299.140324
...,...,...
75,4881.765147,7113.145372
76,1070.275191,1111.171430
77,20376.019465,23976.938955
78,11776.812259,14788.793256


In [76]:
dfMean

,Mean,Shortest Path,optimal shadow path: cost_function_1,optimal shadow path: cost_function_2
0,Shadow part (%),9.956561,24.416956,17.285240
1,Shadow length (m),978.330676,3062.845082,1818.281802
2,Full length (m),10254.031375,12507.313520,10536.963228
3,Ratio shortest/optimal (%),-,0.823975,0.974689


In [137]:
def shortestPathsUnderXkm(y,x):
    srcList = []
    destList =[]
    print("hello")

    while len(srcList) < 5:
        print("hello")
        try:
            src = random.randint(1, 25648) #inclucive
            dest = random.randint(1, 25648)
            path = nx.shortest_path(G, src, dest, weight='full_length')
            path_length = get_total_path_length(path)
            print(path_length)
            if path_length in range(y, x):
                print(path_length)
                print(src)
                print(dest)
                srcList.append(src)
                destList.append(dest)
        except nx.NetworkXNoPath:
            print("No path found :(")
            src = random.randint(1, 25648) #inclucive
            dest = random.randint(1, 25648)
            path = nx.shortest_path(G, src, dest, weight='full_length')
            path_length = get_total_path_length(path)
            
            if path_length in range(y, x):
                print(path_length)
                print(src)
                print(dest)
                srcList.append(src)
                destList.append(dest)
    return srcList, destList


In [130]:
#There is not much difference between shortest paths and optimal paths if they are long
# We only want to consider paths that are <=1km, 2km, 3km... 
# 5 paths for each km --> take the mean 
#First find shortest paths which are <=2km
#Save source and destination and compute there paths with cost functions

srcList1km, destList1km = shortestPathsUnderXkm(0, 1000)

NetworkXNoPath: No path between 23879 and 25555.

In [125]:
srcList2km, destList2km = shortestPathsUnderXkm(1001, 2000)

NetworkXNoPath: No path between 13769 and 8121.

In [128]:
srcList3km, destList3km = shortestPathsUnderXkm(2001, 3000)

NetworkXNoPath: No path between 15674 and 18232.

In [132]:
srcList4km, destList4km = shortestPathsUnderXkm(3001, 4000)


NetworkXNoPath: No path between 4076 and 3352.

In [ ]:
srcList5km, destList5km = shortestPathsUnderXkm(4001, 5000)


In [ ]:
srcList6km, destList6km = shortestPathsUnderXkm(5001, 6000)


In [ ]:
srcList7km, destList7km = shortestPathsUnderXkm(6001, 7000)


In [136]:
srcList8km, destList8km = shortestPathsUnderXkm(7001, 8000)

No path found :(
No path found :(


KeyboardInterrupt: 

In [ ]:
#Get optimal paths
#1km
list_of_paths_shortest = get_80_paths(srcList1km, destList1km, "full_length")
list_of_paths_cost1 = get_80_paths(srcList1km, destList1km, weight_function_1)
list_of_paths_cost2 = get_80_paths(srcList1km, destList1km, weight_function_2)
#2km
list_of_paths_shortest = get_80_paths(srcList2km, destList2km, "full_length")
list_of_paths_cost1 = get_80_paths(srcList2km, destList2km, weight_function_1)
list_of_paths_cost2 = get_80_paths(srcList2km, destList2km, weight_function_2)

#3km
list_of_paths_shortest = get_80_paths(srcList3km, destList3km, "full_length")
list_of_paths_cost1 = get_80_paths(srcList3km, destList3km, weight_function_1)
list_of_paths_cost2 = get_80_paths(srcList3km, destList3km, weight_function_2)
#4km
list_of_paths_shortest = get_80_paths(srcList4km, destList4km, "full_length")
list_of_paths_cost1 = get_80_paths(srcList4km, destList4km, weight_function_1)
list_of_paths_cost2 = get_80_paths(srcList4km, destList4km, weight_function_2)
#5km
list_of_paths_shortest = get_80_paths(srcList5km, destList5km, "full_length")
list_of_paths_cost1 = get_80_paths(srcList5km, destList5km, weight_function_1)
list_of_paths_cost2 = get_80_paths(srcList5km, destList5km, weight_function_2)
#6km
list_of_paths_shortest = get_80_paths(srcList6km, destList6km, "full_length")
list_of_paths_cost1 = get_80_paths(srcList6km, destList6km, weight_function_1)
list_of_paths_cost2 = get_80_paths(srcList6km, destList6km, weight_function_2)
#7km
list_of_paths_shortest = get_80_paths(srcList7km, destList7km, "full_length")
list_of_paths_cost1 = get_80_paths(srcList7km, destList7km, weight_function_1)
list_of_paths_cost2 = get_80_paths(srcList7km, destList7km, weight_function_2)
#8km
list_of_paths_shortest = get_80_paths(srcList8km, destList8km, "full_length")
list_of_paths_cost1 = get_80_paths(srcList8km, destList8km, weight_function_1)
list_of_paths_cost2 = get_80_paths(srcList8km, destList8km, weight_function_2)





In [ ]:

tableMean = [['Mean', 'full_length', 'cost_function_2'], 
         ['Shadow part (%)', mean_part_shortest*100, mean_shadow_part_cost2*100], 
         ['Shadow length (m)', mean_shadow_length_shortest, mean_shadow_length_cost2], 
         ['Full length (m)', mean_total_length_shortest, mean_total_length_cost2]]

print(tabulate(tableMean))


tableStdev = [['Stdev', 'full_length', 'cost_function_2'], 
         ['Shadow part (%)', stdev_part_shortest*100, stdev_part_cost2*100], 
         ['Shadow length (m)', stdev_shadow_length_shortest, stdev_shadow_length_cost2], 
         ['Full length (m)', stdev_total_length_shortest, stdev_total_length_cost2]]

print(tabulate(tableStdev))

In [63]:
domain = np.linspace(-2, 2, 1000) # dividing the distance between -2 and 2 into 1000 points
plt.figure(figsize=(16, 9))
for mu, std in zip(means, std_values):
    # pdf stands for Probability Density Function, which is the plot the probabilities of each range of values
    probabilities = norm.pdf(domain, mu, std)
    plt.plot(domain, probabilities, label=f"$\mu={mu}$\n$\sigma={std}$\n")

plt.legend()
plt.xlabel("Value")
plt.ylabel("Probability")
plt.show()

NameError: name 'means' is not defined

<Figure size 1600x900 with 0 Axes>

In [ ]:
get_shadowed_part_of_path(regular_shortest_path)

In [ ]:
get_shadowed_part_of_path(regular_shortest_path)

In [ ]:
#Testing

# Function for benchmarking a cost function against the regular shortst path with 'full_length' as the weight

def cost_function_performance(src, dst, cost_function):
    regular_shortest_path = nx.shortest_path(G, source=src, target=dst, weight='full_length')
    print("INSIDE COST FUNCTION")
    print()
    optimized_path = nx.shortest_path(G, source=src, target=dst, weight=cost_function)
    print()
    print("OUTSIDE COST FUNCTION")
    print()
    shadowed_part_regular = get_shadowed_part_of_path(regular_shortest_path)
    shadowed_part_optimized = get_shadowed_part_of_path(optimized_path)
    benchmark = shadowed_part_regular
    performance = 100*(shadowed_part_optimized/benchmark)
    print("shadowed_part_regular = ",end='') 
    print(shadowed_part_regular)
    print("shadowed_part_optimized = ",end='')
    print(shadowed_part_optimized)
    print("benchmark = ", end='')
    print(benchmark)
    print("performance = ",end='')
    print(performance)
    return performance


In [ ]:
def cost_function_performance(src, dst, cost_function):
    #calculate shortest path
    regular_shortest_path = nx.shortest_path(G, source=src, target=dst, weight='full_length')
    print("INSIDE COST FUNCTION")
    print()
    optimized_path = nx.shortest_path(G, source=src, target=dst, weight=cost_function)
    print()
    print("OUTSIDE COST FUNCTION")
    print()
    shadowed_part_regular = get_shadowed_part_of_path(regular_shortest_path)
    shadowed_part_optimized = get_shadowed_part_of_path(optimized_path)
    benchmark = shadowed_part_regular
    performance = 100*(shadowed_part_optimized/benchmark)
    #print("shadowed_part_regular = ",end='') 
   # print(shadowed_part_regular)
    print("shadowed_part_optimized = ",end='')
    print(shadowed_part_optimized)
    print("benchmark = ", end='')
    print(benchmark)
    print("performance = ",end='')
    print(performance)
    return performance

In [ ]:
get_total_path_length(regular_shortest_path)

In [ ]:
def get_degree_difference(path1, path2):
    return len(path1)-len(path2)


In [ ]:
print(get_degree_difference(optimal_path_test1,regular_shortest_path))

In [ ]:
#This is a function to get number of nodes that differ between two paths

In [ ]:
#########################################################################################################################################################################################################

In [ ]:
p_u = Point((x_lookup[15633],y_lookup[15633]))
p_v = Point((x_lookup[1],y_lookup[1]))

In [ ]:
x_lookup[]
y_lookup[]

In [ ]:
print(p_u)
print(p_v)

In [ ]:
string = LineString([(-1.2338981,52.9724785),(-1.0771582,53.0405631)])

In [ ]:
print (string)

In [ ]:
LineString([p_u,p_v])

In [ ]:
# Function for extracting the LineString of the shortest path

def get_path_coords(path):
    x_lookup = nx.get_node_attributes(G, "x")
    y_lookup = nx.get_node_attributes(G, "y")
    
    path_nodes_coords = []
    for node in path:
        node_coords = [x_lookup[node],y_lookup[node]]
        path_nodes_coords.append(node_coords)
    return path_nodes_coords

In [ ]:
get_path_coords(test)

In [ ]:
# Function which makes feature collection from a list
def coords_to_geoJson(listCoord):
    coords = []
    for pt in listCoord:
         coords.append((pt[0], pt[1]))


    my_list = LineString(coords)

    my_feature = Feature(geometry=my_list)

    feature_collection = FeatureCollection([my_feature])
    return feature_collection